# 05 - PaDiM Baseline Training (Clean Domain)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/IvanNece/Detection-of-Anomalies-with-Localization/blob/main/notebooks/05_padim_baseline_clean.ipynb)

**Phase 4: Baseline Model Implementation**

---

## 📋 Objectives

1. ✅ Train PaDiM on **Train-clean** split (normal images only)
2. ✅ Use consistent preprocessing with PatchCore
3. ✅ Save trained models for each class
4. ✅ Comprehensive testing and validation

## 🔬 PaDiM Method

**PaDiM** = Probabilistic Anomaly Detection with Multi-scale features

- Extracts multi-scale features from ResNet
- Models normal appearance using Gaussian distributions
- Uses Mahalanobis distance for anomaly scoring
- **No gradient-based training** (statistical approach)

---

## Setup - Mount Drive & Configure Paths

In [ ]:
# ============================================================
# SETUP - Mount Google Drive & Clone Repository
# ============================================================

from google.colab import drive
from pathlib import Path
import sys

# Mount Google Drive
print("Mounting Google Drive...")
drive.mount('/content/drive')
print("Done!\n")

# Clone repository
print("Cloning repository...")
!git clone https://github.com/IvanNece/Detection-of-Anomalies-with-Localization.git 2>/dev/null || echo "Repository already exists"
print("Done!\n")

# Setup paths
PROJECT_ROOT = Path('/content/Detection-of-Anomalies-with-Localization')
sys.path.insert(0, str(PROJECT_ROOT))

print(f"📁 Project root: {PROJECT_ROOT}")
print(f"✓ Path configured")

## Imports

In [ ]:
# Standard library
import json
import time
import zipfile
from typing import Dict

# Deep learning
import torch
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

# Project modules (shared with PatchCore)
from src.utils.reproducibility import set_seed
from src.utils.config import Config
from src.data.transforms import get_clean_transforms
from src.data.anomalib_adapter import load_split_data, create_anomalib_dataloader, get_split_statistics
from src.models.padim_wrapper import PadimWrapper

%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')

print("✓ All modules imported successfully")

## Configuration

In [ ]:
# Load configuration
config = Config.load(PROJECT_ROOT / 'configs' / 'experiment_config.yaml')
set_seed(config.seed)

# Setup
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
CLASSES = config.dataset.classes
BATCH_SIZE = 32
NUM_WORKERS = 2  # Colab-optimized

print("📋 Configuration:")
print(f"  Seed: {config.seed}")
print(f"  Device: {DEVICE}")
print(f"  Classes: {CLASSES}")
print(f"  Image size: {config.dataset.image_size}")
print(f"\n🔧 PaDiM Configuration:")
print(f"  Backbone: {config.padim.backbone}")
print(f"  Layers: {config.padim.layers}")
print(f"  N features: {config.padim.n_features}")
print(f"  Distance: {config.padim.distance_metric}")

## Load Data

In [ ]:
# Load splits (shared with PatchCore - same splits for fair comparison)
splits_path = PROJECT_ROOT / 'data' / 'processed' / 'clean_splits.json'

with open(splits_path, 'r') as f:
    all_splits = json.load(f)

print("✓ Loaded clean splits")
print(f"  Classes: {list(all_splits.keys())}")

# Display split statistics
print("\n📊 Split Statistics:\n")
for class_name in CLASSES:
    split_data = all_splits[class_name]
    train_stats = get_split_statistics(split_data, 'train')
    val_stats = get_split_statistics(split_data, 'val')
    
    print(f"{class_name.upper()}:")
    print(f"  Train: {train_stats['num_samples']} samples (100% normal)")
    print(f"  Val:   {val_stats['num_samples']} samples ({val_stats['num_normal']} normal, {val_stats['num_anomalous']} anomalous)\n")

In [ ]:
# Create transform (shared with PatchCore - same preprocessing)
transform = get_clean_transforms(
    image_size=config.dataset.image_size,
    normalize_mean=config.dataset.normalize.mean,
    normalize_std=config.dataset.normalize.std
)

print(f"✓ Transform initialized:")
print(f"  Size: {config.dataset.image_size}x{config.dataset.image_size}")
print(f"  Normalization: ImageNet statistics")
print(f"  Augmentation: None (deterministic preprocessing)")

## Training Loop

In [ ]:
# Initialize results tracking
MODEL_DIR = PROJECT_ROOT / 'outputs' / 'models'
MODEL_DIR.mkdir(parents=True, exist_ok=True)

training_results = {
    'classes': [],
    'num_train_samples': [],
    'training_time_seconds': [],
    'memory_bank_size_mb': [],
    'model_path': []
}

trained_models = {}

print(f"📁 Model directory: {MODEL_DIR}")

In [ ]:
# Train PaDiM for each class
print("\n" + "="*80)
print("🚀 STARTING TRAINING")
print("="*80 + "\n")

for class_name in CLASSES:
    print(f"\n{'='*80}")
    print(f"🔧 Training PaDiM: {class_name.upper()}")
    print(f"{'='*80}\n")
    
    # 1. Load data
    split_data = all_splits[class_name]
    train_loader = create_anomalib_dataloader(
        split_data=split_data,
        split_type='train',
        transform=transform,
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        shuffle=False,
        pin_memory=True if DEVICE == 'cuda' else False
    )
    
    print(f"✓ Loaded {len(train_loader.dataset)} training samples\n")
    
    # 2. Initialize model
    model = PadimWrapper(
        backbone=config.padim.backbone,
        layers=config.padim.layers,
        n_features=config.padim.n_features,
        image_size=config.dataset.image_size,
        device=DEVICE
    )
    
    # 3. Train
    model.fit(train_loader, verbose=True)
    
    # 4. Save
    model_path = MODEL_DIR / f"padim_{class_name}_clean.pt"
    model.save(model_path, include_stats=True)
    
    # 5. Store results
    training_results['classes'].append(class_name)
    training_results['num_train_samples'].append(model.training_stats['num_samples'])
    training_results['training_time_seconds'].append(model.training_stats['training_time_seconds'])
    training_results['memory_bank_size_mb'].append(model.training_stats['memory_bank_size_mb'])
    training_results['model_path'].append(str(model_path))
    
    trained_models[class_name] = model
    
    print(f"\n✅ Completed: {class_name}")
    print(f"   Model: {model_path.name}")
    print(f"   Time: {model.training_stats['training_time_seconds']:.2f}s")

print(f"\n{'='*80}")
print("🎉 ALL CLASSES TRAINED SUCCESSFULLY!")
print(f"{'='*80}\n")

## Training Results

In [ ]:
# Create results DataFrame
results_df = pd.DataFrame(training_results)

print("📊 Training Summary:\n")
display(results_df)

print(f"\n📈 Overall Statistics:")
print(f"  Total training time: {results_df['training_time_seconds'].sum():.2f}s")
print(f"  Average time per class: {results_df['training_time_seconds'].mean():.2f}s")
print(f"  Total model size: {results_df['memory_bank_size_mb'].sum():.2f} MB")

# Save results
results_path = PROJECT_ROOT / 'outputs' / 'results' / 'padim_training_results_clean.json'
results_path.parent.mkdir(parents=True, exist_ok=True)
with open(results_path, 'w') as f:
    json.dump(training_results, f, indent=2)

print(f"\n✓ Results saved: {results_path.name}")

In [ ]:
# Visualize training results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Training time
axes[0].bar(results_df['classes'], results_df['training_time_seconds'], color='steelblue', alpha=0.8)
axes[0].set_xlabel('Class', fontsize=12)
axes[0].set_ylabel('Training Time (seconds)', fontsize=12)
axes[0].set_title('PaDiM Training Time per Class', fontsize=14, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)
for i, v in enumerate(results_df['training_time_seconds']):
    axes[0].text(i, v + 0.5, f"{v:.1f}s", ha='center', fontweight='bold')

# Memory bank size
axes[1].bar(results_df['classes'], results_df['memory_bank_size_mb'], color='coral', alpha=0.8)
axes[1].set_xlabel('Class', fontsize=12)
axes[1].set_ylabel('Memory Bank Size (MB)', fontsize=12)
axes[1].set_title('PaDiM Memory Usage per Class', fontsize=14, fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)
for i, v in enumerate(results_df['memory_bank_size_mb']):
    axes[1].text(i, v + 1, f"{v:.1f}MB", ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

## Testing and Validation

Comprehensive tests to verify models work correctly

In [ ]:
# TEST 1: Verify all models load correctly
print("="*80)
print("🧪 TEST 1: Model Loading")
print("="*80 + "\n")

all_load_success = True
for class_name in CLASSES:
    model_path = MODEL_DIR / f"padim_{class_name}_clean.pt"
    try:
        test_model = PadimWrapper(device=DEVICE)
        test_model.load(model_path)
        print(f"✓ {class_name}: Loaded successfully")
    except Exception as e:
        print(f"✗ {class_name}: Failed - {e}")
        all_load_success = False

if all_load_success:
    print(f"\n{'='*80}")
    print("✅ TEST 1 PASSED: All models load correctly")
    print("="*80)
else:
    print(f"\n{'='*80}")
    print("❌ TEST 1 FAILED: Some models failed to load")
    print("="*80)

In [ ]:
# TEST 2: Verify predictions work and anomalous > normal
print(f"\n\n{'='*80}")
print("🧪 TEST 2: Prediction Validation")
print("="*80 + "\n")

test_results = []

for class_name in CLASSES:
    print(f"Testing {class_name}...")
    model = trained_models[class_name]
    split_data = all_splits[class_name]
    
    # Create val loader
    val_loader = create_anomalib_dataloader(
        split_data=split_data,
        split_type='val',
        transform=transform,
        batch_size=1,
        num_workers=0,
        shuffle=False,
        pin_memory=False
    )
    
    # Get one normal and one anomalous sample
    normal_score = None
    anomalous_score = None
    
    for batch in val_loader:
        label = batch['label'].item()
        
        if label == 0 and normal_score is None:
            score, _ = model.predict(batch['image'], return_heatmap=False)
            normal_score = score
        elif label == 1 and anomalous_score is None:
            score, _ = model.predict(batch['image'], return_heatmap=False)
            anomalous_score = score
        
        if normal_score is not None and anomalous_score is not None:
            break
    
    # Check result
    passed = anomalous_score > normal_score if (normal_score and anomalous_score) else False
    status = "✓" if passed else "✗"
    
    print(f"  {status} Normal score: {normal_score:.4f}")
    print(f"  {status} Anomalous score: {anomalous_score:.4f}")
    print(f"  {status} Test: {'PASSED' if passed else 'FAILED'}\n")
    
    test_results.append(passed)

if all(test_results):
    print(f"{'='*80}")
    print("✅ TEST 2 PASSED: Anomalous scores > Normal scores for all classes")
    print("="*80)
else:
    print(f"{'='*80}")
    print("⚠️  TEST 2 WARNING: Some anomalous scores not higher than normal")
    print("="*80)

In [ ]:
# TEST 3: Visual validation
print(f"\n\n{'='*80}")
print("🧪 TEST 3: Visual Validation")
print("="*80 + "\n")

TEST_CLASS = 'hazelnut'
print(f"Testing visual predictions for: {TEST_CLASS}\n")

model = trained_models[TEST_CLASS]
split_data = all_splits[TEST_CLASS]

val_loader = create_anomalib_dataloader(
    split_data=split_data,
    split_type='val',
    transform=transform,
    batch_size=1,
    num_workers=0,
    shuffle=False,
    pin_memory=False
)

# Get samples
normal_sample = None
anomalous_sample = None

for batch in val_loader:
    if batch['label'].item() == 0 and normal_sample is None:
        normal_sample = batch
    elif batch['label'].item() == 1 and anomalous_sample is None:
        anomalous_sample = batch
    if normal_sample and anomalous_sample:
        break

# Predict
normal_score, normal_heatmap = model.predict(normal_sample['image'], return_heatmap=True)
anomalous_score, anomalous_heatmap = model.predict(anomalous_sample['image'], return_heatmap=True)

print(f"✓ Predictions generated:")
print(f"  Normal score: {normal_score:.4f}")
print(f"  Anomalous score: {anomalous_score:.4f}\n")

# Denormalize for visualization
def denormalize(tensor, mean, std):
    mean = torch.tensor(mean).view(3, 1, 1)
    std = torch.tensor(std).view(3, 1, 1)
    return torch.clamp(tensor * std + mean, 0, 1)

normal_img = denormalize(normal_sample['image'][0].cpu(), config.dataset.normalize.mean, config.dataset.normalize.std)
normal_img = normal_img.permute(1, 2, 0).numpy()

anomalous_img = denormalize(anomalous_sample['image'][0].cpu(), config.dataset.normalize.mean, config.dataset.normalize.std)
anomalous_img = anomalous_img.permute(1, 2, 0).numpy()

# Plot
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Normal sample
axes[0, 0].imshow(normal_img)
axes[0, 0].set_title(f'Normal Image\nScore: {normal_score:.4f}', fontsize=12, fontweight='bold')
axes[0, 0].axis('off')

if normal_heatmap is not None:
    im1 = axes[0, 1].imshow(normal_heatmap, cmap='jet')
    axes[0, 1].set_title('Anomaly Heatmap', fontsize=12, fontweight='bold')
    axes[0, 1].axis('off')
    plt.colorbar(im1, ax=axes[0, 1], fraction=0.046)
    
    axes[0, 2].imshow(normal_img)
    axes[0, 2].imshow(normal_heatmap, cmap='jet', alpha=0.5)
    axes[0, 2].set_title('Overlay', fontsize=12, fontweight='bold')
    axes[0, 2].axis('off')

# Anomalous sample
axes[1, 0].imshow(anomalous_img)
axes[1, 0].set_title(f'Anomalous Image\nScore: {anomalous_score:.4f}', fontsize=12, fontweight='bold', color='red')
axes[1, 0].axis('off')

if anomalous_heatmap is not None:
    im2 = axes[1, 1].imshow(anomalous_heatmap, cmap='jet')
    axes[1, 1].set_title('Anomaly Heatmap', fontsize=12, fontweight='bold')
    axes[1, 1].axis('off')
    plt.colorbar(im2, ax=axes[1, 1], fraction=0.046)
    
    axes[1, 2].imshow(anomalous_img)
    axes[1, 2].imshow(anomalous_heatmap, cmap='jet', alpha=0.5)
    axes[1, 2].set_title('Overlay', fontsize=12, fontweight='bold')
    axes[1, 2].axis('off')

plt.suptitle(f'PaDiM Predictions - {TEST_CLASS.upper()}', fontsize=16, fontweight='bold', y=0.98)
plt.tight_layout()
plt.show()

print(f"{'='*80}")
print("✅ TEST 3 PASSED: Visualizations generated successfully")
print("="*80)

## Download Results

Create a ZIP package with all outputs for download

In [ ]:
from google.colab import files

print("="*80)
print("📦 Preparing Download Package")
print("="*80 + "\n")

# Create ZIP
zip_path = PROJECT_ROOT / 'padim_outputs.zip'

with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Add models
    for class_name in CLASSES:
        model_file = MODEL_DIR / f"padim_{class_name}_clean.pt"
        stats_file = MODEL_DIR / f"padim_{class_name}_clean.json"
        
        if model_file.exists():
            zipf.write(model_file, f"models/{model_file.name}")
            print(f"✓ Added: {model_file.name}")
        
        if stats_file.exists():
            zipf.write(stats_file, f"models/{stats_file.name}")
            print(f"✓ Added: {stats_file.name}")
    
    # Add results JSON
    if results_path.exists():
        zipf.write(results_path, f"results/{results_path.name}")
        print(f"✓ Added: {results_path.name}")

print(f"\n✅ Package created: {zip_path.name}")
print(f"   Size: {zip_path.stat().st_size / (1024**2):.2f} MB")

# Download
print(f"\n{'='*80}")
print("⬇️  DOWNLOADING...")
print("="*80)
files.download(str(zip_path))
print("\n✅ Download complete!")

---

## Summary

### ✅ Completed Tasks

1. ✓ Trained PaDiM models for 3 classes (hazelnut, carpet, zipper)
2. ✓ Saved models to `outputs/models/padim_{class}_clean.pt`
3. ✓ Generated training statistics and visualizations
4. ✓ Validated models load correctly
5. ✓ Verified predictions work (anomalous > normal scores)
6. ✓ Created visual validation
7. ✓ Downloaded results package

### 📁 Outputs

```
outputs/models/
├── padim_hazelnut_clean.pt      (~50-100 MB each)
├── padim_hazelnut_clean.json
├── padim_carpet_clean.pt
├── padim_carpet_clean.json
├── padim_zipper_clean.pt
└── padim_zipper_clean.json

outputs/results/
└── padim_training_results_clean.json
```

### 🔜 Next Steps

1. Wait for PatchCore completion (other branch)
2. Threshold calibration (notebook 06)
3. Evaluation on clean domain
4. Model comparison (PaDiM vs PatchCore)
5. Domain shift evaluation

---

### 📝 Notes

- This notebook uses **shared components** with PatchCore:
  - `src/data/transforms.py` - Transform classes
  - `src/data/anomalib_adapter.py` - Data adapter
  - `data/processed/clean_splits.json` - Splits
  - `configs/experiment_config.yaml` - Configuration

- Ensure consistency when merging branches for fair model comparison

---

**Status:** ✅ Phase 4 Complete - PaDiM Baseline Ready for Evaluation